In [1]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\imane\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\imane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("movie_review.csv")

In [3]:
df.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos


Pre-processing des donnees

In [4]:
def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Supprimer les stopwords
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return " ".join(filtered_tokens)

# Appliquer le prétraitement à la colonne Text
df['text'] = df['text'].apply(preprocess_text)

# Afficher les premières lignes du dataframe après prétraitement
print(df.head())

   fold_id cv_tag  html_id  sent_id  \
0        0  cv000    29590        0   
1        0  cv000    29590        1   
2        0  cv000    29590        2   
3        0  cv000    29590        3   
4        0  cv000    29590        4   

                                                text  tag  
0  films adapted comic books plenty success wheth...  pos  
1  starters created alan moore eddie campbell bro...  pos  
2  say moore campbell thoroughly researched subje...  pos  
3  book graphic novel 500 pages long includes nea...  pos  
4                     words dont dismiss film source  pos  


Entrainement du modele World2Vec

In [5]:
sentences = [nltk.word_tokenize(text) for text in df['text']]
word2vec_model = Word2Vec(sentences, min_count=1)

vectorisation des reviews de movies

In [6]:
import numpy as np
def vectorize_text(text, model):
    vectors = [model.wv[word] for word in text.split() if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

df['Vectorized_Text'] = df['text'].apply(lambda x: vectorize_text(x, word2vec_model))

Division du DataSet

In [7]:
from sklearn.model_selection import train_test_split
X = np.stack(df['Vectorized_Text'])
y = df['tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train

array([[-0.41827753,  0.33701664,  0.13139227, ..., -0.56660223,
         0.28039196,  0.11181377],
       [-0.19551231,  0.28784049,  0.22536075, ..., -0.32538813,
         0.27042177,  0.15396434],
       [-0.43864599,  0.34217989,  0.2218264 , ..., -0.44466072,
         0.26293349,  0.19951399],
       ...,
       [-0.23715489,  0.34615776,  0.23647138, ..., -0.41680625,
         0.38379627,  0.12918906],
       [-0.31762525,  0.31279024,  0.14273883, ..., -0.44833574,
         0.27026704,  0.08976299],
       [-0.21170072,  0.23350361,  0.11700314, ..., -0.31379968,
         0.16326283,  0.10806384]])

Construction d un classificateur 

In [9]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

C:\Users\imane\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Evaluation du model

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Prédiction sur l'ensemble de test
y_pred = classifier.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)


Accuracy: 0.5705346106304079
Precision: 0.5723909646687068
Recall: 0.5705346106304079
F1 score: 0.5648927782917735
